In [ ]:
import pandas as pd
import numpy as np 
import numpy.random as rd
from sklearn.model_selection import train_test_split
import random

pd.set_option('display.max_columns',None)

In [ ]:
df_home=pd.read_csv('/Users/aurelientarroux/Desktop/Projet_prog/Projet_1/Data/Train_Data/train_home_team_statistics_df.csv')
df_away=pd.read_csv('/Users/aurelientarroux/Desktop/Projet_prog/Projet_1/Data/Train_Data/train_away_team_statistics_df.csv')

df_home.columns='home_'+df_home.columns
df_away.columns='away_'+df_away.columns

df_result=pd.read_csv('/Users/aurelientarroux/Desktop/Projet_prog/Projet_1/Data/Y_train_1rknArQ.csv')

base_match=pd.merge(df_home,df_away,left_on="home_ID",right_on='away_ID',how='inner')
base=pd.merge(base_match,df_result,left_on="home_ID",right_on='ID',how='inner')
base=base.drop(['home_ID','home_LEAGUE','home_TEAM_NAME','away_ID','away_LEAGUE','away_TEAM_NAME'],axis=1)

def resultat(row):
    if row['HOME_WINS']==1:
        return 'HOME'
    elif row['AWAY_WINS']==1:
        return 'AWAY'
    else:
        return 'DRAW'

base['Resultat']=base.apply(resultat, axis=1)
base=base.drop(['HOME_WINS','AWAY_WINS','DRAW'],axis=1)

base.dropna(inplace=True)

X = base.drop(['Resultat','ID'],axis=1)
y = base['Resultat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train

In [ ]:
n_arbre=100
n_sample=len(X)
n_carac=int(np.sqrt(len(X.columns)))
colonne=list(X.columns)

In [ ]:
def calcule_gini(carac,individus):
    liste_Gini=[]
    nb_individus=len(individus)
    for k in list(carac): #je traite le calcule de gini pour chaque caractéristiques 
    
        carac_traite=k
        df_interess=pd.DataFrame(X_train.loc[list(individus),carac])

        #je détermine les différents seuils possible
        a=list(df_interess.loc[:,carac_traite])
        liste_distincte = list(set(a))
        liste_distincte.sort()
        liste_moy=[]
        for j in range(len(liste_distincte)-1):
            liste_moy+= [(liste_distincte[j+1]+liste_distincte[j])/2]

        #je calcule l'indice de Gini de la caractéritsique pour chaque nombre de liste_moy et faut que m'assure que l'index soit l'id match
        for i in liste_moy:

            index_inf=df_interess[df_interess[carac_traite]<i].index
            index_sup=df_interess[df_interess[carac_traite]>i].index

            
            df_inf=y_train.loc[list(index_inf)]
            df_sup=y_train.loc[list(index_sup)]

            nb_inf=(df_inf.value_counts()/len(df_inf))**2
            nb_sup=(df_sup.value_counts()/len(df_sup))**2

            Gini_1=0
            Gini_2=0
            for k in nb_inf:
                Gini_1+=k
            for k in nb_sup:
                Gini_2+=k
            Gini_1=1-Gini_1
            Gini_2=1-Gini_2

            Gini_split= ((len(index_inf)/nb_individus)*Gini_1) + ((len(index_sup)/nb_individus)*Gini_2)
            
            liste_Gini+=[[Gini_split,i,carac_traite]]

    liste_Gini.sort(key=lambda x: x[0])  

    carac_chosie=liste_Gini[0][2]
    seuil=liste_Gini[0][1]

    return  [carac_chosie, seuil]



In [ ]:
colonne=list(X_train.columns)
individus_tot=X_train.index
print(individus_tot)
nombre_arbre=100
foret=[]


for k in range(nombre_arbre):
    
    arbre=[]
    liste_liste_individus=[list(individus_tot)]
    n = len(liste_liste_individus)
    liste_individus = random.choices(liste_liste_individus, k=n)
    
    liste_carac=[[0]]
    while len(liste_individus)>0:
        individus_inter=liste_individus[0]
        
        if len(individus_inter)>10:
            
            carac=rd.choice(colonne, size=n_carac, replace=True)
            carac_choisie , seuil =calcule_gini(carac,individus_inter)
            df_inter=X_train.loc[list(individus_inter),:]
            individus_gauche=df_inter[df_inter[carac_choisie]<seuil].index
            individus_droite=df_inter[df_inter[carac_choisie]>seuil].index
            
            liste_individus+=[list(individus_gauche),list(individus_droite)]
            
            nouvelle_carac_gauche = liste_carac[0] + [[carac_choisie, seuil, '<']]
            nouvelle_carac_droite = liste_carac[0] + [[carac_choisie, seuil, '>']]
            liste_carac += [nouvelle_carac_gauche, nouvelle_carac_droite]
            
        else:
            top_resultat=((y_train.loc[individus_inter].value_counts()).sort_values(ascending=False).index)[0]
            arbre+=[[top_resultat,liste_carac[0]]]

        del liste_individus[0]
        del liste_carac[0]
    
    foret+=[[arbre]]
    print(k)


In [ ]:
colonne=list(X_train.columns)
individus_tot=X_test.index
compte_ratio_final=0

for k in range(len(individus_tot)):
    liste_resultat_final=[]
    individus_test=X_test.iloc[k,:]
    for arbre in foret:
        for branche in arbre[0]:
            resultat=branche[0]
            compte_bon=0
            compte_neoud=0
            
            for neoud in branche[1]:
                if neoud!=0:
                    compte_neoud+=1
                    
                    carac=neoud[0]
                    seuil=neoud[1]
                    signe=neoud[2]
                    
                    carac_individus=individus_test[carac]
                    
                    if signe=='<':
                        if carac_individus<seuil:
                            compte_bon+=1
                    
                    elif signe=='>':
                        if carac_individus>seuil:
                            compte_bon+=1
                    else:
                        print('merde')
            
            if compte_bon==compte_neoud:
                liste_resultat_final+=[resultat]
                break

    if max(liste_resultat_final, key=liste_resultat_final.count)==y_test.iloc[k]:
        compte_ratio_final+=1
    print(compte_ratio_final/(k+1))


In [ ]:
print(compte_ratio_final)